<a href="https://colab.research.google.com/github/praveen555/IMI-BIGDATA--CHALLENGE/blob/main/baseline_experiment_02_new_features_ensemble_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.metrics import *

In [2]:
#np.random.seed(0) #Set to compare the difference across various experiements and reproduce the same results.

In [3]:
data=pd.read_csv('clean_uoft_nodes_v2.csv',
                 parse_dates=['BIRTH_DT','CUST_ADD_DT'])

In [4]:
col=['CASH_CNT_IN','CASH_SUM_OUT','CASH_SUM_IN','CASH_CNT_OUT', 'WIRES_SUM_IN', 'WIRES_CNT_IN',
       'WIRES_SUM_OUT', 'WIRES_CNT_OUT']

In [5]:
# ## Some the features have 0 and give inf or Nan when we divide
for i in col:
  data[i]=data[i]+1

In [6]:
data['cash_diff']=data['CASH_SUM_IN']-data['CASH_SUM_OUT']
data['wire_diff']=data['WIRES_SUM_IN']-data['WIRES_SUM_OUT']
data['cash_cnt_diff']=data['CASH_CNT_IN']-data['CASH_CNT_OUT']
data['wire_cnt_diff']=data['WIRES_CNT_IN']-data['WIRES_CNT_OUT']
data['avg_cash_transaction_out']=data['CASH_SUM_OUT']/data['CASH_CNT_OUT']
data['avg_wire_transaction_out']=data['WIRES_SUM_OUT']/data['WIRES_CNT_OUT']
data['avg_cash_transaction_in']=data['CASH_SUM_IN']/data['CASH_CNT_IN']
data['avg_wire_transaction_in']=data['WIRES_SUM_IN']/data['WIRES_CNT_IN']


In [7]:
data.head()

,Unnamed: 0,BIRTH_DT,CUST_ADD_DT,OCPTN_NM,RES_CNTRY_CA,CNTRY_OF_INCOME_CA,PEP_FL,CASH_SUM_IN,CASH_CNT_IN,CASH_SUM_OUT,...,GENDER,CUSTOMER_ID,cash_diff,wire_diff,cash_cnt_diff,wire_cnt_diff,avg_cash_transaction_out,avg_wire_transaction_out,avg_cash_transaction_in,avg_wire_transaction_in
0,0,1981-09-01,2007-07-05,High,1,1,0.0,2578.785,3.0,2852.663,...,Female,935382.0,-273.878,46530.0,-5.0,27.0,356.582875,673.472222,859.595000,1123.412698
1,1,1994-02-21,2019-05-19,High,1,1,0.0,3036.502,3.0,4806.997,...,Male,2305.0,-1770.495,86490.0,-13.0,-76.0,300.437313,1283.921569,1012.167333,3674.415584
2,2,1962-11-16,2011-08-02,High,1,1,0.0,1618.571,2.0,3483.809,...,Male,472403.0,-1865.238,-25966.0,-10.0,-14.0,290.317417,1599.952381,809.285500,1090.428571
3,3,1998-06-20,2001-08-15,High,1,1,0.0,3588.042,6.0,1941.943,...,Female,227994.0,1646.099,43357.0,-3.0,-167.0,215.771444,443.083682,598.007000,2072.972222
4,4,1942-01-24,2012-10-28,High,1,1,0.0,1726.524,3.0,13198.169,...,Female,418265.0,-11471.645,-95085.0,-41.0,-84.0,299.958386,1035.381215,575.508000,951.742268


In [8]:
data.columns

Index(['Unnamed: 0', 'BIRTH_DT', 'CUST_ADD_DT', 'OCPTN_NM', 'RES_CNTRY_CA',
       'CNTRY_OF_INCOME_CA', 'PEP_FL', 'CASH_SUM_IN', 'CASH_CNT_IN',
       'CASH_SUM_OUT', 'CASH_CNT_OUT', 'WIRES_SUM_IN', 'WIRES_CNT_IN',
       'WIRES_SUM_OUT', 'WIRES_CNT_OUT', 'COUNTRY_RISK_INCOME',
       'COUNTRY_RISK_RESIDENCY', 'RISK', 'NAME', 'GENDER', 'CUSTOMER_ID',
       'cash_diff', 'wire_diff', 'cash_cnt_diff', 'wire_cnt_diff',
       'avg_cash_transaction_out', 'avg_wire_transaction_out',
       'avg_cash_transaction_in', 'avg_wire_transaction_in'],
      dtype='object')

In [9]:
data=data[data.columns[1:]]

In [10]:
data.drop(['BIRTH_DT','CUST_ADD_DT','CUSTOMER_ID','NAME'],inplace=True,axis=1)

In [11]:
data.head()

,OCPTN_NM,RES_CNTRY_CA,CNTRY_OF_INCOME_CA,PEP_FL,CASH_SUM_IN,CASH_CNT_IN,CASH_SUM_OUT,CASH_CNT_OUT,WIRES_SUM_IN,WIRES_CNT_IN,...,RISK,GENDER,cash_diff,wire_diff,cash_cnt_diff,wire_cnt_diff,avg_cash_transaction_out,avg_wire_transaction_out,avg_cash_transaction_in,avg_wire_transaction_in
0,High,1,1,0.0,2578.785,3.0,2852.663,8.0,70775.0,63.0,...,low,Female,-273.878,46530.0,-5.0,27.0,356.582875,673.472222,859.595000,1123.412698
1,High,1,1,0.0,3036.502,3.0,4806.997,16.0,282930.0,77.0,...,low,Male,-1770.495,86490.0,-13.0,-76.0,300.437313,1283.921569,1012.167333,3674.415584
2,High,1,1,0.0,1618.571,2.0,3483.809,12.0,7633.0,7.0,...,low,Male,-1865.238,-25966.0,-10.0,-14.0,290.317417,1599.952381,809.285500,1090.428571
3,High,1,1,0.0,3588.042,6.0,1941.943,9.0,149254.0,72.0,...,low,Female,1646.099,43357.0,-3.0,-167.0,215.771444,443.083682,598.007000,2072.972222
4,High,1,1,0.0,1726.524,3.0,13198.169,44.0,92319.0,97.0,...,low,Female,-11471.645,-95085.0,-41.0,-84.0,299.958386,1035.381215,575.508000,951.742268


In [12]:
data['RISK'].value_counts()


low       600000
medium    350000
high       50000
Name: RISK, dtype: int64

## Biased only 5%  is high risk

## One hot encode

In [13]:
data=pd.get_dummies(data,columns=['OCPTN_NM','COUNTRY_RISK_RESIDENCY', 'GENDER','COUNTRY_RISK_INCOME'])

In [14]:
data.head()

,RES_CNTRY_CA,CNTRY_OF_INCOME_CA,PEP_FL,CASH_SUM_IN,CASH_CNT_IN,CASH_SUM_OUT,CASH_CNT_OUT,WIRES_SUM_IN,WIRES_CNT_IN,WIRES_SUM_OUT,...,OCPTN_NM_Low,OCPTN_NM_Moderate,COUNTRY_RISK_RESIDENCY_High,COUNTRY_RISK_RESIDENCY_Low,COUNTRY_RISK_RESIDENCY_Moderate,GENDER_Female,GENDER_Male,COUNTRY_RISK_INCOME_High,COUNTRY_RISK_INCOME_Low,COUNTRY_RISK_INCOME_Moderate
0,1,1,0.0,2578.785,3.0,2852.663,8.0,70775.0,63.0,24245.0,...,0,0,0,1,0,1,0,0,1,0
1,1,1,0.0,3036.502,3.0,4806.997,16.0,282930.0,77.0,196440.0,...,0,0,0,1,0,0,1,0,1,0
2,1,1,0.0,1618.571,2.0,3483.809,12.0,7633.0,7.0,33599.0,...,0,0,0,1,0,0,1,0,1,0
3,1,1,0.0,3588.042,6.0,1941.943,9.0,149254.0,72.0,105897.0,...,0,0,0,1,0,1,0,0,1,0
4,1,1,0.0,1726.524,3.0,13198.169,44.0,92319.0,97.0,187404.0,...,0,0,0,1,0,1,0,0,1,0


## Split in train-test

In [15]:
y=data['RISK']

In [16]:
x=data.loc[:, data.columns !='RISK']

In [17]:
x.head()

,RES_CNTRY_CA,CNTRY_OF_INCOME_CA,PEP_FL,CASH_SUM_IN,CASH_CNT_IN,CASH_SUM_OUT,CASH_CNT_OUT,WIRES_SUM_IN,WIRES_CNT_IN,WIRES_SUM_OUT,...,OCPTN_NM_Low,OCPTN_NM_Moderate,COUNTRY_RISK_RESIDENCY_High,COUNTRY_RISK_RESIDENCY_Low,COUNTRY_RISK_RESIDENCY_Moderate,GENDER_Female,GENDER_Male,COUNTRY_RISK_INCOME_High,COUNTRY_RISK_INCOME_Low,COUNTRY_RISK_INCOME_Moderate
0,1,1,0.0,2578.785,3.0,2852.663,8.0,70775.0,63.0,24245.0,...,0,0,0,1,0,1,0,0,1,0
1,1,1,0.0,3036.502,3.0,4806.997,16.0,282930.0,77.0,196440.0,...,0,0,0,1,0,0,1,0,1,0
2,1,1,0.0,1618.571,2.0,3483.809,12.0,7633.0,7.0,33599.0,...,0,0,0,1,0,0,1,0,1,0
3,1,1,0.0,3588.042,6.0,1941.943,9.0,149254.0,72.0,105897.0,...,0,0,0,1,0,1,0,0,1,0
4,1,1,0.0,1726.524,3.0,13198.169,44.0,92319.0,97.0,187404.0,...,0,0,0,1,0,1,0,0,1,0


## First split dataset and ensure test set. 

idea is generate multiple weak learnes trained on different subsets of data and combine them using Voting classifier.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=42)

In [19]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(700000, 30) (300000, 30) (700000,) (300000,)


In [20]:
y_test.value_counts()

low       180212
medium    104916
high       14872
Name: RISK, dtype: int64

In [21]:
y_train.value_counts()

low       419788
medium    245084
high       35128
Name: RISK, dtype: int64

## Now only work with x_train and y_train and generate multiple subsets for classifier to learn on.

## 1st subset - train on all unbalanced dataset

In [22]:
model_1=HistGradientBoostingClassifier()
model_1.fit(X_train,y_train)

HistGradientBoostingClassifier()

In [23]:
model_6=RandomForestClassifier()
model_6.fit(X_train,y_train)

RandomForestClassifier()

2nd subset- Undersample to have balanced dataset 

## Undersample the x_train

In [24]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled=rus.fit_resample(X_train,y_train)

In [25]:
X_resampled.head()

,RES_CNTRY_CA,CNTRY_OF_INCOME_CA,PEP_FL,CASH_SUM_IN,CASH_CNT_IN,CASH_SUM_OUT,CASH_CNT_OUT,WIRES_SUM_IN,WIRES_CNT_IN,WIRES_SUM_OUT,...,OCPTN_NM_Low,OCPTN_NM_Moderate,COUNTRY_RISK_RESIDENCY_High,COUNTRY_RISK_RESIDENCY_Low,COUNTRY_RISK_RESIDENCY_Moderate,GENDER_Female,GENDER_Male,COUNTRY_RISK_INCOME_High,COUNTRY_RISK_INCOME_Low,COUNTRY_RISK_INCOME_Moderate
0,1,1,0.0,16365.490,12.0,13602.0,13.0,1286954.0,17.0,229268.0,...,1,0,0,1,0,0,1,0,1,0
1,1,1,1.0,31321.497,10.0,19724.0,6.0,2083754.0,223.0,1211917.0,...,1,0,0,1,0,0,1,0,1,0
2,1,1,0.0,33934.835,16.0,18619.0,12.0,1135212.0,118.0,287657.0,...,1,0,0,1,0,1,0,0,1,0
3,1,1,0.0,13293.952,6.0,646.0,1.0,3228217.0,37.0,1616665.0,...,0,0,0,1,0,0,1,0,1,0
4,1,1,1.0,66062.641,32.0,37513.0,49.0,3934182.0,168.0,1976339.0,...,1,0,0,1,0,1,0,0,1,0


In [26]:
y_resampled.value_counts()

high      35128
low       35128
medium    35128
Name: RISK, dtype: int64

In [27]:
model_2=HistGradientBoostingClassifier()
model_2.fit(X_resampled,y_resampled)

HistGradientBoostingClassifier()

In [28]:
model_7=RandomForestClassifier()
model_7.fit(X_resampled,y_resampled)

RandomForestClassifier()

3rd subset of data- Train only on low and medium

In [29]:
low_medium_index=y_train[(y_train=='low')|(y_train=='medium')].index

In [30]:
low_medium_y_train=y_train[(y_train=='low')|(y_train=='medium')]

In [31]:
low_medium_x_train=X_train.loc[low_medium_index]

In [32]:
print(low_medium_x_train.shape,low_medium_y_train.shape)

(664872, 30) (664872,)


In [33]:
model_3=HistGradientBoostingClassifier()
model_3.fit(low_medium_x_train,low_medium_y_train)

HistGradientBoostingClassifier()

In [34]:
model_8=RandomForestClassifier()
model_8.fit(low_medium_x_train,low_medium_y_train)

RandomForestClassifier()

## 4th subset of data is high-medium

In [35]:
high_medium_index=y_train[(y_train=='high')|(y_train=='medium')].index
high_medium_y_train=y_train[(y_train=='high')|(y_train=='medium')]
high_medium_x_train=X_train.loc[high_medium_index]
print(high_medium_x_train.shape,high_medium_y_train.shape)

(280212, 30) (280212,)


In [36]:
model_4=HistGradientBoostingClassifier()
model_4.fit(high_medium_x_train,high_medium_y_train)

HistGradientBoostingClassifier()

In [37]:
model_9=RandomForestClassifier()
model_9.fit(high_medium_x_train,high_medium_y_train)

RandomForestClassifier()

## 5th type of dataset high-low

In [38]:
high_low_index=y_train[(y_train=='high')|(y_train=='low')].index
high_low_y_train=y_train[(y_train=='high')|(y_train=='low')]
high_low_x_train=X_train.loc[high_low_index]
print(high_low_x_train.shape,high_low_y_train.shape)

(454916, 30) (454916,)


In [39]:
model_5=HistGradientBoostingClassifier()
model_5.fit(high_low_x_train,high_low_y_train)

HistGradientBoostingClassifier()

In [40]:
model_10=RandomForestClassifier()
model_10.fit(high_low_x_train,high_low_y_train)

RandomForestClassifier()

## Standarize the datasets- Not required on tree based algos.

In [41]:
#from sklearn.preprocessing import StandardScaler


In [42]:
# scaler = StandardScaler()
# normalized_x_train = pd.DataFrame(
#     scaler.fit_transform(X_resampled),
#     columns = X_resampled.columns, index=X_resampled.index)

In [43]:
# normalized_x_train.head()

In [44]:
# normalized_x_test = pd.DataFrame(scaler.transform(test_t),columns = test_t.columns,index=test_t.index)

In [45]:
# normalized_x_test.head()

In [46]:
# ## helper function 
# def reports(y_true,y_pred):
#   print("Accuracy: ",accuracy_score(y_true,y_pred))
#   print("Recall Score: ",recall_score(y_true,y_pred,average='macro'))
#   print("Precision Score: ",precision_score(y_true,y_pred,average='macro'))
#   print("F1 score: ",f1_score(y_true,y_pred,average='macro'))
#   print(classification_report(y_true,y_pred))
#   print("Confusion Matrix : ")
#   print(confusion_matrix(y_true,y_pred))

In [47]:
# models= [('LogReg',LogisticRegression()),
         
#           ('RF', RandomForestClassifier()),
#           ('AdaBoostClassifier',AdaBoostClassifier() ),
#           ('GradientBoostingClassifier',GradientBoostingClassifier()), 
#           ('HistGradientBoostingClassifier',HistGradientBoostingClassifier()),
#         ]

In [48]:
# def experiment(x_train,x_test,y_train,y_test):
#   for name,model in models:
#     clf = model.fit(x_train, y_train)
#     y_pred = clf.predict(x_test)
#     print("For model: ",name)
#     reports(y_test,y_pred)


In [49]:
# experiment(normalized_x_train,normalized_x_test,y_resampled,y_test)

tp-13658,
fn-1242,fp-21450
tn-263650
tp+fp=

hist gradient boosting classier works best out of models selected. Better to chose higher recall than precision 

Question is what kind of error could be tolerate ? 

<li> Classifying an innocent customer as fraud (low precision)
<li> Missing to identifying as someone who made fraud (low recall)

<li>Recall is Fraud identified / Over total Fraud 
<li>Precision is “fraud caught over predictions I made for fraud”

From confusion matrix the model made lot of false positive (fp='fill value') as high risk customers which would be bad customer experience

However on other hand from test set the model was able to correctly predict (tp='fill value') high risk customers over 15k cases. So a high recall is achieved. 


Some other notes

1. Balancing the dataset helped improved the score. 
2. HP tuning to imporve the results further ? 
3. Try feature engineering 
4. Try deep learning networks to improve this score. 

In [50]:
## 11th dataset
high_index=y_train[y_train=='high'].index
high_y_train=y_train[y_train=='high']
high_x_train=X_train.loc[high_index]
print(high_x_train.shape,high_y_train.shape)


(35128, 30) (35128,)


In [51]:
model_11=RandomForestClassifier()
model_11.fit(high_x_train,high_y_train)

RandomForestClassifier()

## Voting Classifier

In [52]:
pip install mlxtend  


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
pip install mlxtend --upgrade --no-deps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [54]:
# import six
# import sys
# sys.modules['sklearn.externals.six'] = six

In [55]:
from mlxtend.classifier import EnsembleVoteClassifier
import copy

In [56]:
eclf = EnsembleVoteClassifier(clfs=[model_1,model_2,model_3,model_4,model_5],fit_base_estimators=False)


In [57]:
eclf.fit(X_train,y_train)

/usr/local/lib/python3.8/dist-packages/mlxtend/classifier/ensemble_vote.py:173: UserWarning: fit_base_estimators=False enforces use_clones to be `False`
  warnings.warn(


EnsembleVoteClassifier(clfs=[HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier()],
                       fit_base_estimators=False, use_clones=False)

In [58]:
y_pred_eclf=eclf.predict(X_test)

In [59]:
print(classification_report(y_test,y_pred_eclf))

              precision    recall  f1-score   support

        high       0.63      0.48      0.54     14872
         low       1.00      1.00      1.00    180212
      medium       0.93      0.96      0.94    104916

    accuracy                           0.96    300000
   macro avg       0.85      0.81      0.83    300000
weighted avg       0.96      0.96      0.96    300000



In [60]:
eclf2= EnsembleVoteClassifier(clfs=[model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9,model_10],fit_base_estimators=False)


In [61]:
eclf2.fit(X_train,y_train)

/usr/local/lib/python3.8/dist-packages/mlxtend/classifier/ensemble_vote.py:173: UserWarning: fit_base_estimators=False enforces use_clones to be `False`
  warnings.warn(


EnsembleVoteClassifier(clfs=[HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier(),
                             RandomForestClassifier(), RandomForestClassifier(),
                             RandomForestClassifier(), RandomForestClassifier(),
                             RandomForestClassifier()],
                       fit_base_estimators=False, use_clones=False)

In [62]:
y_pred_eclf2=eclf2.predict(X_test)

In [63]:
print(classification_report(y_test,y_pred_eclf2))

              precision    recall  f1-score   support

        high       0.60      0.53      0.56     14872
         low       1.00      1.00      1.00    180212
      medium       0.93      0.95      0.94    104916

    accuracy                           0.96    300000
   macro avg       0.85      0.83      0.83    300000
weighted avg       0.96      0.96      0.96    300000



In [64]:
model_7.feature_importances_

array([8.35907720e-04, 4.44564075e-04, 9.58847809e-03, 5.03782263e-02,
       1.52304243e-02, 4.31734127e-02, 1.53443108e-02, 1.40813677e-01,
       1.63070169e-02, 7.08373738e-02, 1.72460770e-02, 4.00749415e-02,
       8.21745974e-02, 1.52815982e-02, 1.67091964e-02, 1.44969634e-01,
       7.73126438e-02, 7.48538180e-02, 1.58231021e-01, 1.99208471e-03,
       2.16103734e-03, 5.26496821e-04, 6.09338021e-04, 6.62628653e-04,
       8.89558369e-05, 1.82255571e-03, 1.79537327e-03, 2.34336037e-04,
       2.53438955e-04, 4.68353565e-05])

In [65]:
model_7.feature_names_in_

array(['RES_CNTRY_CA', 'CNTRY_OF_INCOME_CA', 'PEP_FL', 'CASH_SUM_IN',
       'CASH_CNT_IN', 'CASH_SUM_OUT', 'CASH_CNT_OUT', 'WIRES_SUM_IN',
       'WIRES_CNT_IN', 'WIRES_SUM_OUT', 'WIRES_CNT_OUT', 'cash_diff',
       'wire_diff', 'cash_cnt_diff', 'wire_cnt_diff',
       'avg_cash_transaction_out', 'avg_wire_transaction_out',
       'avg_cash_transaction_in', 'avg_wire_transaction_in',
       'OCPTN_NM_High', 'OCPTN_NM_Low', 'OCPTN_NM_Moderate',
       'COUNTRY_RISK_RESIDENCY_High', 'COUNTRY_RISK_RESIDENCY_Low',
       'COUNTRY_RISK_RESIDENCY_Moderate', 'GENDER_Female', 'GENDER_Male',
       'COUNTRY_RISK_INCOME_High', 'COUNTRY_RISK_INCOME_Low',
       'COUNTRY_RISK_INCOME_Moderate'], dtype=object)

In [66]:
feature_importance=pd.DataFrame({'feature':model_7.feature_names_in_,'score':model_7.feature_importances_})

In [67]:
feature_importance.sort_values(['score'],inplace=True,ascending=False)

In [68]:
feature_importance

,feature,score
18,avg_wire_transaction_in,0.158231
15,avg_cash_transaction_out,0.144970
7,WIRES_SUM_IN,0.140814
12,wire_diff,0.082175
16,avg_wire_transaction_out,0.077313
17,avg_cash_transaction_in,0.074854
9,WIRES_SUM_OUT,0.070837
3,CASH_SUM_IN,0.050378
5,CASH_SUM_OUT,0.043173
11,cash_diff,0.040075


In [69]:
eclf3= EnsembleVoteClassifier(clfs=[model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9,model_10,model_11],fit_base_estimators=False)


In [70]:
eclf3.fit(X_train,y_train)

/usr/local/lib/python3.8/dist-packages/mlxtend/classifier/ensemble_vote.py:173: UserWarning: fit_base_estimators=False enforces use_clones to be `False`
  warnings.warn(


EnsembleVoteClassifier(clfs=[HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier(),
                             HistGradientBoostingClassifier(),
                             RandomForestClassifier(), RandomForestClassifier(),
                             RandomForestClassifier(), RandomForestClassifier(),
                             RandomForestClassifier(),
                             RandomForestClassifier()],
                       fit_base_estimators=False, use_clones=False)

In [71]:
y_pred_eclf3=eclf3.predict(X_test)

In [72]:
print(classification_report(y_test,y_pred_eclf3))

              precision    recall  f1-score   support

        high       0.60      0.53      0.56     14872
         low       1.00      1.00      1.00    180212
      medium       0.93      0.95      0.94    104916

    accuracy                           0.96    300000
   macro avg       0.85      0.83      0.83    300000
weighted avg       0.96      0.96      0.96    300000

